Lib imports

In [11]:
from gx_utils import *
import logging
import numpy as np
import random
from random import seed, choice
from typing import Callable

logging.basicConfig(format="%(message)s", level=logging.INFO)

Problem definition - lab delivery

In [12]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

Professor's solution - lab delivery

In [61]:
def greedy(N):
    goal = set(range(N))
    covered = set()
    solution = list()
    all_lists = sorted(problem(N, seed=42), key=lambda l: len(l))
    print(all_lists, " all lists")
    while goal != covered:
        x = all_lists.pop(0)
        print(x, " x")
        if not set(x) < covered:
            solution.append(x)
            covered |= set(x)

    logging.info(
        f"Greedy solution for N={N}: w={sum(len(_) for _ in solution)} (bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )
    logging.debug(f"{solution}")
greedy(5)

INFO:root:Greedy solution for N=5: w=5 (bloat=0%)


[[0], [1], [0], [4], [0], [1], [4], [4], [4], [2], [1], [0], [3], [3], [4], [0], [1], [3], [1, 3], [0, 1], [0, 2], [2, 4], [2, 4], [0, 1], [2, 3]]  all lists
[0]  x
[1]  x
[0]  x
[4]  x
[0]  x
[1]  x
[4]  x
[4]  x
[4]  x
[2]  x
[1]  x
[0]  x
[3]  x


In [14]:
logging.getLogger().setLevel(logging.INFO)
for N in [5, 10, 20, 100, 500, 1000]:
    greedy(N)

INFO:root:Greedy solution for N=5: w=5 (bloat=0%)
INFO:root:Greedy solution for N=10: w=13 (bloat=30%)
INFO:root:Greedy solution for N=20: w=46 (bloat=130%)
INFO:root:Greedy solution for N=100: w=332 (bloat=232%)
INFO:root:Greedy solution for N=500: w=2162 (bloat=332%)
INFO:root:Greedy solution for N=1000: w=4652 (bloat=365%)


In [ ]:
%timeit greedy(1_000)


Search algorithm

In [23]:
class State:
    def __init__(self, data: np.ndarray):
        self._data = data.copy()
        self._data.flags.writeable = False

    def __hash__(self):
        return hash(bytes(self._data))

    def __eq__(self, other):
        return bytes(self._data) == bytes(other._data)

    def __lt__(self, other):
        return bytes(self._data) < bytes(other._data)

    def __str__(self):
        return str(self._data)

    def __repr__(self):
        return repr(self._data)

    @property
    def data(self):
        return self._data

    def copy_data(self):
        return self._data.copy()

In [37]:
def search(
    initial_state: State,
    goal_test: Callable,
    parent_state: dict,
    state_cost: dict,
    priority_function: Callable,
    unit_cost: Callable,
):
    frontier = PriorityQueue()
    parent_state.clear()
    state_cost.clear()

    state = initial_state
    parent_state[state] = None
    state_cost[state] = 0

    while state is not None and not goal_test(state):
        for a in possible_actions(state):
            new_state = result(state, a)
            cost = unit_cost(a)
            if new_state not in state_cost and new_state not in frontier:
                parent_state[new_state] = state
                state_cost[new_state] = state_cost[state] + cost
                frontier.push(new_state, p=priority_function(new_state))
                logging.debug(f"Added new node to frontier (cost={state_cost[new_state]})")
            elif new_state in frontier and state_cost[new_state] > state_cost[state] + cost:
                old_cost = state_cost[new_state]
                parent_state[new_state] = state
                state_cost[new_state] = state_cost[state] + cost
                logging.debug(f"Updated node cost in frontier: {old_cost} -> {state_cost[new_state]}")
        if frontier:
            state = frontier.pop()
        else:
            state = None

    path = list()
    s = state
    while s:
        path.append(s.copy_data())
        s = parent_state[s]

    logging.info(f"Found a solution in {len(path):,} steps; visited {len(state_cost):,} states")
    return list(reversed(path))

GOAL E INITIAL STATE

In [54]:
GOAL = (np.array(range(5)))
logging.info(f"Goal:\n{GOAL}")
print(type(GOAL))

def goal_test(state):
    return state == GOAL

INFO:root:Goal:
[0 1 2 3 4]


<class 'numpy.ndarray'>


In [ ]:
INITIAL_STATE = problem(5,42)
print(len(INITIAL_STATE)) #quante parentesi
print((INITIAL_STATE))

In [70]:
def set_covering_problem(universe,subsets,costs):
    cost=0
    visited=0
    #create a unique set - source: documentation
    elements=set(e for s in subsets for e in s) 
    if elements!=universe:
        return None
    covered=set()
    solution=[]
    while covered!=elements:
        subset = max(subsets,key=lambda s: len(s-covered)/costs[subsets.index(s)])
        solution.append(subset)
        cost+=costs[subsets.index(subset)]
        visited=visited+1
        covered |= subset
    print("VISITED NODES: ",visited)
    return solution,cost 

def start(N):
    """ Start: 
        N: N
        I suppose cost are unit
    """
    sub=problem(N,seed=42)
    #all costs are 1 
    C=np.full(len(sub),1)
    print("C ",C)
    #with array i got errors
    universe=set(range(N))
    #create a unique set - source: documentation
    subsets=[set(element) for element in sub]
    costs=C 
    set_cover,totalCosts=set_covering_problem(universe,subsets,costs)
    print("UNIVERSE: ",universe)
    print("COVERING SETS: ",set_cover)
    print("COST: ",totalCosts)

#N=5
print("5")
start(5)
print("----------------------------------------------------------------")
print("10")
start(10)
print("----------------------------------------------------------------")
print("20")
start(20)
print("----------------------------------------------------------------")
print("50")
start(50)
print("----------------------------------------------------------------")
print("100")
start(100)



5
C  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
VISITED NODES:  3
UNIVERSE:  {0, 1, 2, 3, 4}
COVERING SETS:  [{1, 3}, {0, 2}, {4}]
COST:  3
----------------------------------------------------------------
10
C  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
VISITED NODES:  3
UNIVERSE:  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
COVERING SETS:  [{0, 3, 4, 7, 9}, {8, 1, 6}, {2, 5, 7}]
COST:  3
----------------------------------------------------------------
20
C  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
VISITED NODES:  4
UNIVERSE:  {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
COVERING SETS:  [{0, 3, 5, 8, 9, 10, 13, 14, 17}, {4, 7, 11, 12, 15, 16, 18}, {0, 1, 2, 3, 6, 13, 17, 18}, {0, 6, 16, 17, 19}]
COST:  4
----------------------------------------------------------------
50
C  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 